In [1]:
import plumed
import matplotlib.pyplot as plt
import os
import MDAnalysis

/home/lammps/.conda/envs/plumed-masterclass/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set working directory for all future cells
os.chdir("/home/lammps/plumed/PRL-2017-PairEntropy/Project/Proj23")
# and check with bash command pwd - note the exclamation mark at the beginning
!pwd
# or in python
os.getcwd()

/home/lammps/plumed/PRL-2017-PairEntropy/Project/Proj23


'/home/lammps/plumed/PRL-2017-PairEntropy/Project/Proj23'

In [51]:
%%bash
# create PLUMED input file
cat > "start.lmp" << EOF

variable    p_id world   0 # 1 2 3 4 5
log         log.lammps.\${p_id} append


#temperature 
variable    temperature equal 350.
variable    tempDamp equal 0.1 # approx 0.1 ps

#pressure
variable    pressure equal 1.
variable    pressureDamp equal 10.0

#seed
variable    seed world 74581 # 93734 12832 21934 57383 49172
units       metal
atom_style  full
box         tilt large


#box
variable side equal 22.0
variable        numAtoms equal 250
variable        mass equal 22.98977
region          box block 0 \${side} 0 \${side} 0 \${side}
create_box      1 box
create_atoms    1 random \${numAtoms} \${seed} box
mass            1 \${mass}
change_box      all triclinic


# #setup
variable        out_freq equal 500
variable        out_freq2 equal 5000

neigh_modify    delay 10 every 1
pair_style      eam/fs
pair_coeff      * * Na_MendelevM_2014.eam.fs Na


timestep        0.002 # According to Frenkel and Smit is 0.001

thermo          \${out_freq}
thermo_style    custom step temp pe press lx ly lz xy xz yz pxx pyy pzz pxy pxz pyz

restart         \${out_freq2} restart.\${p_id} restart2.\${p_id}


minimize 1.0e-2 1.0e-3 100 1000

reset_timestep  0

# NVT

fix             1 all nve
fix             2 all temp/csvr \${temperature} \${temperature} \${tempDamp} \${seed}

velocity        all create \${temperature} \${seed} dist gaussian
run             25000

unfix           1
unfix           2
EOF

In [52]:
%%bash
lmp -in start.lmp

LAMMPS (30 Jul 2021)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Created orthogonal box = (0.0000000 0.0000000 0.0000000) to (22.000000 22.000000 22.000000)
  1 by 1 by 1 MPI processor grid
Created 250 atoms
  using lattice units in orthogonal box = (0.0000000 0.0000000 0.0000000) to (22.000000 22.000000 22.000000)
  create_atoms CPU = 0.001 seconds
Changing box ...
  triclinic box = (0.0000000 0.0000000 0.0000000) to (22.000000 22.000000 22.000000) with tilt (0.0000000 0.0000000 0.0000000)
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 11.2
  ghost atom cutoff = 11.2
  binsize = 5.6, bins = 4 4 4
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/fs, perpetual
      attributes: half, newton on
      pair build: half/bin/newton/tri
      stencil: half/bin/3d/tri
      bin: standard
Setting

   12000    358.64072   -257.05514   -1600.7883           22           22           22            0            0            0    -949.8931   -1725.0764   -2127.3954   -379.57513    764.05378   -263.19569 
   12500    357.85301   -256.90834   -1544.9658           22           22           22            0            0            0   -1165.4639   -1832.1187   -1637.3148    348.28507    236.23979   -125.77995 
   13000    357.78481   -256.12406   -1328.4832           22           22           22            0            0            0    -967.8655    -1465.868    -1551.716   -61.560714    512.60529   -12.586245 
   13500    391.87517    -256.0304   -1189.3251           22           22           22            0            0            0   -1211.0253   -1863.0236   -493.92648    347.30586   -42.164859   -4.5055829 
   14000    343.05443   -257.10976   -1618.7324           22           22           22            0            0            0   -912.38212    -1899.261   -2044.5541    239.91962   